In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - 稳定扩散XL 1.0 - TPU v5e

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_jax_stable_diffusion_xl.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_jax_stable_diffusion_xl.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_jax_stable_diffusion_xl.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
在Vertex AI Workbench中打开
    </a>
    (建议使用预装的HuggingFace/transformer库的Python-3 GPU笔记本)
  </td>
</table>

## 概述

本笔记本演示如何在Vertex AI上部署[stabilityai/stable-diffusion-xl-base-1.0](https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0)模型进行在线预测。

### 目标

- 将模型部署到[Vertex AI Endpoint资源](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints)。
- 运行文本到图像的在线预测。

### 成本

本教程使用Google Cloud的付费组件：

* Vertex AI
* 云存储

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)和[云存储价格](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

## 开始之前

**注意**:

* Jupyter运行以`!`为前缀的行作为shell命令，并将以`$`为前缀的Python变量插入这些命令中。
* 本笔记本演示了如何在Vertex AI预测端点上部署模型[stabilityai/stable-diffusion-xl-base-1.0](https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0)，使用TPU v5e实例（机器类型为`ct5lp-hightpu-1t`）。请确保您在`us-west1`地区有足够的资源配额。如果没有，请按照[说明](https://cloud.google.com/vertex-ai/docs/predictions/use-tpu#securing_capacity)获取配额。

### 设置笔记本

#### Colab
在Colab中运行以下命令。

In [ ]:
if "google.colab" in str(get_ipython()):
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

# Restart the notebook kernel after installs.
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### 设置谷歌云项目

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建账户时，您将获得$300的免费信用用于计算/存储成本。

2. [确保您的项目已启用结算](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API 和 Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

4. [创建一个云存储桶](https://cloud.google.com/storage/docs/creating-buckets) 用于存储实验输出。

5. [创建一个服务账号](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console) 并分配 `Vertex AI 用户` 和 `存储对象管理员` 角色，以便部署模型到 Vertex AI 终点。

设置实验环境的以下变量:

In [1]:
# Cloud project id.
PROJECT_ID = ""  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = ""  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output. Fill it without the 'gs://' prefix.
GCS_BUCKET = ""  # @param {type:"string"}

# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = ""  # @param {type:"string"}

初始化 Vertex AI API:

In [2]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

### 定义常量

In [3]:
# The pre-built serving docker image. It contains serving scripts and models.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/jax-diffusers-serve-tpu:20240110_1526_RC00"

###定义常见的函数

In [6]:
import base64
from io import BytesIO

from google.cloud import aiplatform
from PIL import Image


def base64_to_image(image_str):
    """Convert base64 encoded string to an image."""
    image = Image.open(BytesIO(base64.b64decode(image_str)))
    return image


def image_grid(imgs, rows=2, cols=2):
    w, h = imgs[0].size
    grid = Image.new(
        mode="RGB", size=(cols * w + 10 * cols, rows * h), color=(255, 255, 255)
    )
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w + 10 * i, i // cols * h))
    return grid


def deploy_model(model_id):
    """Create a Vertex AI Endpoint and deploy the specified model to the endpoint."""
    model_name = model_id + "-tpu"
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")

    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[8080],
        serving_container_predict_route="/predict",
        serving_container_health_route="/health",
    )
    machine_type = "ct5lp-hightpu-1t"

    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        deploy_request_timeout=1800,
        service_account=SERVICE_ACCOUNT,
        enable_access_logging=True,
        min_replica_count=1,
        sync=True,
    )
    return model, endpoint

上传和部署模型

这个部分将模型上传到模型注册表，并将其部署到Vertex AI终端资源。

模型部署步骤将需要大约30分钟的时间才能完成。

文本到图像

部署稳定的diffusion xl模型用于文本到图像的任务。

一旦部署完成，您可以将一批文本提示发送到端点生成图像。

在一个TPU V5e实例上部署时，一个图像的平均推理时间约为3秒。

In [ ]:
# Set the model_id to "stabilityai/stable-diffusion-xl-base-1.0" to load the OSS pre-trained model.
model, endpoint = deploy_model(
    model_id="stabilityai/stable-diffusion-xl-base-1.0",
)

In [ ]:
instances = [
    {
        "prompt": "Photorealistic whale swimming in abyss",
        "height": 1024,
        "width": 1024,
    },
    {
        "prompt": "Photorealistic happy dog running",
        "height": 1024,
        "width": 1024,
    },
]
response = endpoint.predict(instances=instances)

images = [
    base64_to_image(prediction.get("images")[0]) for prediction in response.predictions
]
image_grid(images, rows=1)

清理资源：

In [ ]:
# Undeploy model and delete endpoint.
endpoint.delete(force=True)

# Delete models.
model.delete()